<a href="https://colab.research.google.com/github/moafar/pai_ssab/blob/master/check_pai4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

'''
v4.0 - 30/08/2023

Este script recorre el directorio de trabajo y procesa los archivos .XLSM que allí se encuentren.

El procesamiento consiste en la validación de requisitos obligatorios, definidos en errores tipo:
  "error2": "Faltan datos en columnas obligatorias",
  "error3": "Faltan datos en columnas obligatorias para menores de 18",
  "error4": "Faltan datos de madre o cuidador en menores de 18",
  "error5": "Datos incompletos de la madre en menores de 18 ",
  "error6": "Datos incompletos del cuidador en menores de 18 ",
  "error7a": "Datos incompletos en gestantes - FUR",
  "error7b": "Datos incompletos en gestantes - Embarazos previos",
  "error8a": "Datos incompletos en antecedentes <<contraindicado>>",
  "error8b": "Datos incompletos en antecedentes <<Evento adverso>> ",
  "error9": "Datos incompletos en vacunas "

'''

import os
import pandas as pd
import datetime
import openpyxl
import warnings

# Ignorar el mensaje acerca de los campos validados del Excel
warnings.filterwarnings("ignore", message="Data Validation extension is not supported and will be removed")

columnas_obligatorias = {
    1:"Fecha de atención formato de fecha en números (día/mes/año)*",
    2:"Tipo de identificación*",
    3:"Número de identificación*",
    4:"Primer nombre*",
    6:"Primer apellido*",
    8:"Fecha de nacimiento Formato de fecha en números (día/mes/año)*",
    14:"Sexo*",
    15:"Género",
    16:"Orientación sexual",
    18:"País de nacimiento*",
    21:"Régimen de afiliación*",
    22:"Aseguradora *",
    23:"Pertenencia étnica*",
    24:"Desplazado*",
    25:"Discapacitado*",
    26:"Fallecido*",
    27:"Víctima del conflicto armado*",
    28:"Estudia actualmente*",
    29:"País de residencia*",
    30:"Departamento de residencia*",
    31:"Municipio de residencia*",
    33:"Área*",
    38:"¿Autoriza llamadas teléfonicas? *",
    39:"¿Autoriza envío de correo? *",
    40:"ANTEC-MD-¿Sufre o ha sufrido algún evento o enfermedad que contraindique la vacunación?*",
    42:"ANTEC-MD-¿Ha presentado reacción moderada o severa a biológicos anteriores?*",
    252:"RESPONSABLE (NOMBRE DEL VACUNADOR)"
}

columnas_obligatorias_18 = {
    13: "Esquema completo",
    17: "Edad gestacional (semanas)",
    20: "Lugar de atención del parto* (Hospital)"
}

columnas_datos_madre = {
    53:"MADRE-Tipo de identificación",
    54:"MADRE-Número de identificación*",
    55:"MADRE-Primer nombre*",
    57:"MADRE-Primer apellido*",
    59:"MADRE-Correo electrónico*",
    62:"MADRE-Régimen de afiliación*",
    63:"MADRE-Pertenencia étnica*",
    64:"MADRE-Desplazado*"
}

columnas_datos_cuidador = {
    65:"CUIDADOR-Tipo de identificación",
    66:"CUIDADOR-Número de identificación*",
    67:"CUIDADOR-Primer nombre*",
    69:"CUIDADOR-Primer apellido*",
    71:"CUIDADOR-Parentesco*",
    72:"CUIDADOR-Correo electrónico*"
}

# Defino paquetes de condiciones para evaluar datos por vacuna
conjuntos = {
    'covid': [76, 77, 78, 79, 80, 81],
    'BCG': [82, 83, 84, 85, 86, 87],
    'HepB': [88, 89, 90, 91, 92],
    'PolioInactIny': [93, 94, 95, 96, 97],
    'PolioOral': [98, 99, 100],
    'Penta': [101, 102, 103, 104, 105],
    'Hexa': [106, 107, 108, 109],
    'DPT': [110, 111, 112, 113],
    'DPTaPed': [114, 115, 116, 117],
    'TD_Ped': [118, 119, 120, 121],
    'RotavOral': [122, 123],
    'Neumococo': [124, 125, 126, 127, 128],
    'SRP': [129, 130, 131, 132, 133],
    'SR_Multidosis': [134, 135, 136, 137, 138],
    'Famarilla': [139, 140, 141, 142, 143],
    'HepA_Ped': [144, 145, 146, 147],
    'Varic': [148, 149, 150, 151, 152],
    'TTyD_Adulto': [153, 154, 155, 156],
    'dTpa_Adulto': [157, 158, 159, 160],
    'Influenza': [161, 162, 163, 164, 165],
    'VPH': [166, 167, 168, 169],
    'Antirr_Vac': [170, 171, 172, 173, 174, 175],
    'Antirr_Suero': [176, 177],
    'HepB_Inmunog': [178, 179, 180, 181, 182],
    'IG_ATT_SueroHomo': [183, 184, 185, 186],
    'Antit_Dift_SueroHetero': [187, 188, 189, 190],
    'Meningococo': [191, 192, 193, 194, 195],
    'HepB': [196, 197],
    'PentaV': [198, 199],
    'HexaV': [200, 201],
    'TetraV': [202, 203],
    'DPT_Acel_Ped': [204, 205],
    'TTyD_Ped': [206, 207],
    'Rotav': [208, 209],
    'Neumoc_Conj': [210, 211],
    'Neumoc_Polisac': [212, 213],
    'Tripleviral': [214, 215],
    'Varicela_Tripleviral': [216, 217],
    'Famarilla': [218, 219],
    'HepA': [220, 221],
    'HepA,HepB': [222, 223],
    'Varic': [224, 225],
    'TT_Antidift_Adult': [226, 227],
    'DPT_Acel_Adult': [228, 229],
    'Influenza': [230, 231],
    'VPH': [232, 233],
    'Antirr_Profilac': [234, 235, 236],
    'IG_Antitet_SueroHomo': [237, 238],
    'IG_AntihepB': [239, 240, 241],
    'IG_Antitet_SueroHomo': [242, 243],
    'Antitox_Tet_SueroHetero': [244, 245],
    'MeningoConj': [246, 247],
    'FiebreTifo': [248, 249],
    'Hzoster': [250, 251]
}

nombre_hoja = "Registro Diario"

# Obtener la ruta del directorio actual (Get Current Working Directory)
# o sea, el directorio de este archivo .py
directorio_actual = os.getcwd()

# Obtener una lista de los archivos y directorios en el directorio actual
lista_archivos = os.listdir(directorio_actual)

for archivo in lista_archivos:
    ruta_archivo = os.path.join(directorio_actual, archivo)
    if os.path.isfile(ruta_archivo) and archivo.endswith(".xlsm"):
      print("Procesando: ", archivo)

      df = pd.read_excel(archivo, header=None, skiprows=2, sheet_name="Registro Diario")
      df = df[df.iloc[:, 1].notnull()]

      # Eliminar la última fila (con valores "fin")
      df = df.drop(df.index[-1])

      # Subset de menores de 18 años
      df[df.columns[9]] = pd.to_numeric(df[df.columns[9]], errors='coerce')
      df18 = df[df[df.columns[9]] < 18]

      error0 = False # Disponible
      error1 = False # Disponible

      # Validación columnas obligatorias (error2)
      # *************************************************************************
      print("Evaluación campos obligatorios...")

      error2 = False # Columnas obligatorias

      for col in columnas_obligatorias:     # Valido cada columna de la lista de obligatorias
        if df.loc[:, col].isnull().any():   # Si alguna tiene algún valor vacío, da un error.
          print("Hay vacío en la columna " + str(col) + ": " + columnas_obligatorias[col])
          error2 = True
      print("...............................")
      print()
      print("Validación de errores:")

      if error2:
        print("* Error2 positivo")
      else:
        print("Aprobado sin error 2")

      # Validación datos obligatorios para menores de 18 (error3)
      # *************************************************************************

      error3 = False # Columnas obligatorias para menores de 18

      for col in columnas_obligatorias_18:      # Valido cada columna de la lista (obligatorias para < 18)
          if df18.loc[:, col].isnull().any():   # Si la columna tiene algún vacío da un error.
            print("Error en la columna: " + str(col) + " (Vacíos en campos requeridos para <18)")
            error3 = True

      if error3:
        print("* Error3 positivo")
      else:
        print("Aprobado sin error 3")

      # Validación datos de madre o cuidador (error4)
      # *************************************************************************

      error4 = False # Presencia de datos de madre o cuidador en menores de 18

      for idx, row in df18.iterrows():
          if pd.isna(row[54]):    # Si la ID de la  madre está vacía...
            if pd.isna(row[66]):  # Verifica si la ID del cuidador está vacía también.
              error4 = True          # Eso da un error

      if error4:
        print("* Error4 positivo")
      else:
        print("Aprobado sin error 4")

      # Validación  datos obligatorios de la madre (error5)
      # *************************************************************************

      error5 = False # Datos completos de la madre en menores de 18

      for idx, row in df18.iterrows():
        if pd.notna(row[54]):  # Si la ID de la madre está diligenciada...
          if any(pd.isna(row[col]) for col in columnas_datos_madre):  # Verifica que todos los datos obligatorios madre estén
            error5 = True

      if error5:
        print("* Error5 positivo")
      else:
        print("Aprobado sin error 5")

      # Validación datos obligatorios del cuidador (error6)
      # *************************************************************************

      error6 = False # Datos completos del cuidador en menores de 18

      for idx, row in df18.iterrows():
        if pd.notna(row[66]): # Si existe la ID del cuidador...
          if any(pd.isna(row[col]) for col in columnas_datos_cuidador):  # Verifica que los demás datosdel cuidador existan también
            error6 = True

      if error6:
        print("* Error6 positivo")
      else:
        print("Aprobado sin error 6")

      # Validación datos obligatorios de gestantes (error7) -- VERSION 2
      # *************************************************************************

      error7a = False # Datos completos en gestantes (Fecha ultima regla)
      error7b = False # Datos completos en gestantes (Numero de embarazos previos)

      dfG = df[df[44]=="GESTANTE"]

      columnas_gestante = {
          45: "USUARIA-Gestante Fecha de última menstruación",
          48: "USUARIA-Cantidad de embarazos previos"
      }

      if pd.isna(dfG[45]).any():
        error7a = True

      if error7a:
        print("Error7a positivo")
      else:
        print("Aprobado sin error 7a")


      if pd.isna(dfG[48]).any():
        error7b = True

      if error7b:
        print("Error7b positivo")
      else:
        print("Aprobado sin error 7b")

      # Validación antecedentes (error8)
      # *************************************************************************

      error8a = False # Datos completos en antecedentes "contraindicado" y "Evento adverso"
      error8b = False # Datos completos en antecedentes "contraindicado" y "Evento adverso"

      # Verifica que los antecedentes sean coherentes (si antecedente contraindicado es SI, debe estar diligenciado CUAL)
      dfContra = df[df[40] == "SI"]

      if pd.isna(dfContra[41]).any():
          error8a = True

      if error8a:
        print("Error8a positivo")
      else:
        print("Aprobado sin error 8a")


      # Verifica que los antecedentes sean coherentes (si evento adverso previo es SI, debe decir CUAL)
      dfAdverso = df[df[42] == "SI"]

      if pd.isna(dfAdverso[41]).any():
          error8b = True

      if error8b:
        print("Error8b positivo")
      else:
        print("Aprobado sin error 8b")

      # Validación de datos completos en las vacunas (error9)
      # *************************************************************************

      error9 = False # Datos incompletos en vacunas


      resultados_por_fila = {}

      for nombre_conjunto, columnas in conjuntos.items():
          mask = df[columnas].notnull().any(axis=1) & df[columnas].isnull().any(axis=1)

          # Filtrar las filas con True en la máscara
      filas_con_errores = df[mask]

      if not filas_con_errores.empty:
          error9 = True

      if error9:
        print("* Error9 positivo")
      else:
        print("Aprobado sin error 9")

      # Imprimir resultados para el conjunto actual
      if not filas_con_errores.empty:
          for index, row in filas_con_errores.iterrows():
              if index not in resultados_por_fila:
                  resultados_por_fila[index] = []
              resultados_por_fila[index].append(nombre_conjunto)

      # Ordenar las filas por su índice
      filas_ordenadas = sorted(resultados_por_fila.keys())

      # Imprimir los resultados agrupados y ordenados por fila
      for index in filas_ordenadas:
          conjuntos_con_error = resultados_por_fila[index]
          mensaje = f'Fila {index+1}: Datos incompletos en {", ".join(conjuntos_con_error)}'
          print(mensaje)

      print()

      errores = {
          "error2": error2,
          "error3": error3,
          "error4": error4,
          "error5": error5,
          "error6": error6,
          "error7a": error7a,
          "error7b": error7b,
          "error8a": error8a,
          "error8b": error8b,
          "error9": error9
      }

      error_descriptions = {
          "error2": "Faltan datos en columnas obligatorias",
          "error3": "Faltan datos en columnas obligatorias para menores de 18",
          "error4": "Faltan datos de madre o cuidador en menores de 18",
          "error5": "Datos incompletos de la madre en menores de 18 ",
          "error6": "Datos incompletos del cuidador en menores de 18 ",
          "error7a": "Datos incompletos en gestantes - FUR",
          "error7b": "Datos incompletos en gestantes - Embarazos previos",
          "error8a": "Datos incompletos en antecedentes <<contraindicado>>",
          "error8b": "Datos incompletos en antecedentes <<Evento adverso>> ",
          "error9": "Datos incompletos en vacunas "
      }

      print("***************************************")
      print("R E S U L T A D O   D E :", archivo)


      for error_name, error_value in errores.items():
          if error_value:
              error_description = error_descriptions.get(error_name, "Descripción no disponible")
              print(f"Archivo rechazado debido a: {error_description}")

      print("***************************************")

print(" - - - - T E R M I N A D O - - - - ")


Procesando:  SANITAS.xlsm
Evaluación campos obligatorios...
Hay vacío en la columna 6: Primer apellido*
...............................

Validación de errores:
* Error2 positivo
Aprobado sin error 3
* Error4 positivo
* Error5 positivo
* Error6 positivo
Aprobado sin error 7a
Aprobado sin error 7b
Aprobado sin error 8a
Aprobado sin error 8b
Aprobado sin error 9

***************************************
R E S U L T A D O   D E : SANITAS.xlsm
Archivo rechazado debido a: Faltan datos en columnas obligatorias
Archivo rechazado debido a: Faltan datos de madre o cuidador en menores de 18
Archivo rechazado debido a: Datos incompletos de la madre en menores de 18 
Archivo rechazado debido a: Datos incompletos del cuidador en menores de 18 
***************************************
Procesando:  COMFENALCO.xlsm
Evaluación campos obligatorios...
Hay vacío en la columna 22: Aseguradora *
...............................

Validación de errores:
* Error2 positivo
Error en la columna: 17 (Vacíos en campos 